In [1]:
import sys
import cv2
import torch
import numpy as np
from PyQt6.QtWidgets import QApplication, QLabel, QPushButton, QVBoxLayout, QWidget
from PyQt6.QtGui import QImage, QPixmap
from PyQt6.QtCore import QTimer
from ultralytics import YOLO

class DroneUI(QWidget):
    def __init__(self):
        super().__init__()
        
        # Load YOLO model
        self.model = YOLO("yolov8n.pt")
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)
        
        # OpenCV Video Capture
        self.cap = cv2.VideoCapture(0)  # Change index if needed
        
        # UI Components
        self.video_label = QLabel(self)
        #self.ndvi_button = QPushButton("Toggle NDVI Mode", self)
        #self.ndvi_button.clicked.connect(self.toggle_ndvi)
        
        self.vari_button = QPushButton("Toggle VARI Mode", self)
        self.vari_button.clicked.connect(self.toggle_vari)

        # Legend Label
        self.legend_label = QLabel(self)
        self.update_legend()

        # Explanation Labels
        #self.ndvi_explanation = QLabel("NDVI (Normalized Difference Vegetation Index) measures plant health using Near-Infrared (NIR) and Red light.", self)
        self.vari_explanation = QLabel("VARI (Visible Atmospherically Resistant Index) measures vegetation using only visible light, making it useful for RGB cameras.", self)
        self.health_explanation = QLabel("\nLow values (red) = Unhealthy vegetation\nHigh values (green) = Healthy vegetation", self)
        
        # Layout
        layout = QVBoxLayout()
        layout.addWidget(self.video_label)
        #layout.addWidget(self.ndvi_button)
        layout.addWidget(self.vari_button)
        layout.addWidget(self.legend_label)
        #layout.addWidget(self.ndvi_explanation)
        layout.addWidget(self.vari_explanation)
        layout.addWidget(self.health_explanation)
        self.setLayout(layout)
        
        # Timer for updating frames
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_frame)
        self.timer.start(30)  # 30ms delay for ~30 FPS
        
        #self.ndvi_mode = False  # NDVI mode toggle
        self.vari_mode = False  # VARI mode toggle
    
    def toggle_vari(self):
        self.vari_mode = not self.vari_mode  # Switch VARI mode
        #if self.vari_mode:
            #self.ndvi_mode = False  # Ensure only one mode is active
        self.update_legend()

    def update_legend(self):
        legend = np.zeros((100, 256, 3), dtype=np.uint8)  # Increased height for labels
        
        # Create color gradient from red (-1) to green (1)
        for i in range(256):
            ratio = i / 255  # Ratio from 0 to 1
            red = int(255 * (1 - ratio))  # Red decreases from 255 to 0
            green = int(255 * ratio)  # Green increases from 0 to 255
            blue = 0
            color = (blue, green, red)  # BGR format: blue is 0, green increases, red decreases
            legend[:, i] = color  # Apply color to the column
        
        # Add the labels for VARI Scale and the Low/High values
        cv2.putText(legend, "VARI Scale", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
        cv2.putText(legend, "Low", (5, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        cv2.putText(legend, "High", (220, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

        # Add intermediate labels from -1 to 1 on the legend (adjusting for the new color scale)
        value = -1.0
        cv2.putText(legend, f"{value:.1f}", (5, 75), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
            
        # For 0.0 (center)
        value = 0.0
        cv2.putText(legend, f"{value:.1f}", (120, 75), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
            
        # For 1.0 (rightmost)
        value = 1.0
        cv2.putText(legend, f"{value:.1f}", (225, 75), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)


        # Convert to RGB for Qt display
        legend = cv2.cvtColor(legend, cv2.COLOR_BGR2RGB)
        qimg = QImage(legend.data, 256, 100, 256 * 3, QImage.Format.Format_RGB888)  # Adjusted for height
        self.legend_label.setPixmap(QPixmap.fromImage(qimg))



    
    def update_frame(self):
        ret, frame = self.cap.read()
        if not ret:
            return
        
        # Run YOLO object detection
        results = self.model(frame)
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])  
                conf = box.conf[0].item()  
                label = result.names[int(box.cls[0])]  
                
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"{label}: {conf:.2f}", (x1, y1 - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Detect oranges
        orange_count, frame_with_oranges = self.detect_oranges(frame)
        
        # Display orange count on the frame
        cv2.putText(frame_with_oranges, f"Oranges Detected: {orange_count}", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        
        # Apply NDVI or VARI if enabled
        #if self.ndvi_mode:
            #frame_with_oranges = self.apply_ndvi(frame_with_oranges)
        if self.vari_mode:
            frame_with_oranges = self.apply_vari(frame_with_oranges)
        
        # Convert frame to Qt format for display
        frame_with_oranges = cv2.cvtColor(frame_with_oranges, cv2.COLOR_BGR2RGB)
        h, w, ch = frame_with_oranges.shape
        bytes_per_line = ch * w
        qimg = QImage(frame_with_oranges.data, w, h, bytes_per_line, QImage.Format.Format_RGB888)
        self.video_label.setPixmap(QPixmap.fromImage(qimg))
    
    def detect_oranges(self, frame):
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        lower_orange = np.array([5, 150, 150])
        upper_orange = np.array([15, 255, 255])
        mask = cv2.inRange(hsv, lower_orange, upper_orange)
        kernel = np.ones((5, 5), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        orange_count = 0
        for contour in contours:
            if cv2.contourArea(contour) > 500:
                x, y, w, h = cv2.boundingRect(contour)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                orange_count += 1
        return orange_count, frame

    #def apply_ndvi(self, frame):
       # nir = frame[:, :, 2].astype(float)
       # red = frame[:, :, 0].astype(float)
       # ndvi = (nir - red) / (nir + red + 1e-5)
       # ndvi = ((ndvi + 1) / 2) * 255
       # return cv2.applyColorMap(ndvi.astype(np.uint8), cv2.COLORMAP_JET)
    
    def apply_vari(self, frame):
        green = frame[:, :, 1].astype(float)
        red = frame[:, :, 0].astype(float)
        blue = frame[:, :, 2].astype(float)
        vari = (green - red) / (green + red - blue + 1e-5)
        vari = ((vari + 1) / 2) * 255
        return cv2.applyColorMap(vari.astype(np.uint8), cv2.COLORMAP_JET)
    
    def add_legend(self, frame, mode):
        legend = np.zeros((100, 256, 3), dtype=np.uint8)
        for i in range(256):
            legend[:, i] = cv2.applyColorMap(np.array([[i]], dtype=np.uint8), cv2.COLORMAP_JET)
        
        cv2.putText(legend, f"{mode} Scale", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
        cv2.putText(legend, "Low", (5, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        cv2.putText(legend, "High", (220, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        frame[:100, :256] = legend
    
    def closeEvent(self, event):
        self.cap.release()
        cv2.destroyAllWindows()
        event.accept()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = DroneUI()
    window.setWindowTitle("Agricultural Drone UI")
    window.show()
    sys.exit(app.exec())


0: 480x640 1 person, 40.2ms
Speed: 3.0ms preprocess, 40.2ms inference, 69.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 6.2ms
Speed: 1.0ms preprocess, 6.2ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 5.0ms
Speed: 1.0ms preprocess, 5.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 6.0ms
Speed: 2.0ms preprocess, 6.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 5.0ms
Speed: 1.0ms preprocess, 5.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 7.0ms
Speed: 1.0ms preprocess, 7.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 11.0ms
Speed: 2.0ms preprocess, 11.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0:

SystemExit: 0

c:\Users\vardh\miniconda3\envs\cuda_test\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
